# Analyze results for 3D CGAN
Feb 22, 2021

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


In [7]:
img_size=128

In [8]:
val_data_dict={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing',
              '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube'}

### Read validation data

In [9]:
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use

bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=range(len(sigma_lst))
bkgnd_dict={}
num_bkgnd=100

for label in labels_lst:
    fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
    del samples

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_1.1_train_val.npy


## Read data

In [10]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [11]:
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/',
      '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [12]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202106*')]
n=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(n)

interactive(children=(Dropdown(description='x', options=('20210627_130046_cgan_128_nodes1_lr0.00002_vary', '20…

In [17]:
result=n.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210626_100420_cgan-128_bs8_nodes1_lr0.0002


## Plot Losses

In [18]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [19]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
19413,19413.0,82.0,1.817583,0.542101,2.359684,0.815914,inf,inf,-1.619572,inf,-2.745346,NaN,NaN,-1.728930,-4.040251,-0.187703,1.511066,0.0002,0.0002
19414,19414.0,82.0,0.692860,0.849952,1.542812,1.775577,inf,inf,-2.736663,inf,-2.094010,NaN,NaN,2.295391,0.348150,-1.566412,1.518623,0.0002,0.0002
19415,19415.0,82.0,0.439995,0.606914,1.046908,4.457101,inf,inf,-2.883697,inf,-2.862807,NaN,NaN,1.985936,-0.270292,-4.393980,1.523559,0.0002,0.0002
19416,19416.0,82.0,1.925612,0.424349,2.349962,2.633573,inf,inf,-2.663792,inf,-2.764309,NaN,NaN,-1.772376,-3.622752,-2.472544,1.526383,0.0002,0.0002
19417,19417.0,82.0,0.989311,0.361970,1.351281,0.329946,inf,inf,-2.740322,inf,-2.575344,NaN,NaN,-0.321522,-1.518641,1.187213,1.520082,0.0002,0.0002
19418,19418.0,82.0,0.559560,1.322211,1.881771,1.281307,inf,inf,-2.111244,inf,-2.798738,NaN,NaN,2.966251,1.168638,-0.898124,1.518919,0.0002,0.0002
19419,19419.0,82.0,0.439822,0.485846,0.925668,4.954922,inf,inf,-2.546100,inf,-1.889650,NaN,NaN,1.429786,-0.456365,-4.917825,1.526231,0.0002,0.0002
19420,19420.0,82.0,2.016974,0.823647,2.840621,3.218011,14.979589,11.761578,-0.768355,inf,-2.495763,NaN,NaN,-1.908220,-3.833968,-3.128207,1.519298,0.0002,0.0002
19421,19421.0,82.0,1.072649,0.380781,1.453430,0.661896,9.182669,8.520773,-0.813512,inf,-0.729634,NaN,NaN,-0.527433,-1.958917,0.250944,1.509369,0.0002,0.0002
19422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf,-0.779842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [21]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>1)].sort_values(by=['hist_chi']).head(10)

-2.684844493865967


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
15154,15154.0,64.0,0.620804,0.593519,1.214322,4.456707,inf,inf,-1.822320,inf,-3.138103,NaN,NaN,0.942766,-0.292900,-4.434565,1.514671,0.0002,0.0002
15875,15875.0,67.0,0.772749,1.029155,1.801904,2.913945,inf,inf,-2.826562,inf,-3.122404,NaN,NaN,3.303345,0.494187,-2.810307,1.523718,0.0002,0.0002
16691,16691.0,71.0,0.338101,1.326066,1.664167,3.955594,inf,inf,-2.619259,inf,-3.121992,NaN,NaN,3.239818,0.670675,-3.906098,1.518501,0.0002,0.0002
14915,14915.0,63.0,0.442466,1.078636,1.521102,3.404439,inf,inf,-2.512251,inf,-3.120711,NaN,NaN,3.432035,0.958549,-3.310844,1.516369,0.0002,0.0002
16018,16018.0,68.0,0.438383,0.670856,1.109239,2.845403,inf,inf,-2.340101,inf,-3.116860,NaN,NaN,1.946895,-0.642334,-2.760020,1.519275,0.0002,0.0002
14912,14912.0,63.0,1.702218,0.390934,2.093153,1.986877,12.121234,10.134357,-2.066691,inf,-3.116561,NaN,NaN,-1.767323,-3.071848,-1.764100,1.511849,0.0002,0.0002
9195,9195.0,39.0,0.389414,0.752578,1.141991,2.242752,12.655371,10.412619,-1.431645,inf,-3.111020,NaN,NaN,2.151043,-0.076975,-2.065860,1.514410,0.0002,0.0002
16314,16314.0,69.0,0.921849,0.254534,1.176382,2.532649,inf,inf,-2.995256,inf,-3.106344,NaN,NaN,0.174102,-2.020537,-2.302201,1.524512,0.0002,0.0002
14668,14668.0,62.0,0.799548,0.336235,1.135783,0.980829,inf,inf,-3.034786,inf,-3.105772,NaN,NaN,-0.137815,-2.251509,-0.475662,1.523394,0.0002,0.0002
15168,15168.0,64.0,0.455014,0.603550,1.058564,2.540267,inf,inf,-3.056538,inf,-3.104299,NaN,NaN,0.986851,-1.117045,-2.367640,1.524243,0.0002,0.0002


In [22]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

<!-- ### Read validation data -->

## Read stored chi-squares for images

In [29]:
## Get sigma list from saved files
flist=glob.glob(result_dir+'/df_processed*')
sigma_lst=[i.split('/')[-1].split('df_processed_')[-1].split('.pkle')[0] for i in flist]
sigma_lst.sort() ### Sorting is important for labels to match !!

labels_lst=np.arange(len(sigma_lst))

In [30]:
sigma_lst,labels_lst

(['0.5', '0.8', '1.1'], array([0, 1, 2]))

In [31]:
### Create a merged dataframe

df_list=[]
for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(sigma_lst[label])))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_1b']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list


In [32]:
df

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,...,chi_spec3,num_imgs,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_sdev,label
0,0,10,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.331658,0.027100,0.000133,0.358891,0.003603,0.310804,...,65.608524,8,0,0,"[0.6967160468309673, 0.018782222245251817, 0.0...","[0.0003101189048411497, 0.0016778901227197975,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[197598462.26789847, 32421950.699822515, 14711...","[21009315.769309275, 3425476.9592361706, 14776...",0-10
1,0,20,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.323898,0.027100,0.000133,0.351131,0.003530,0.223106,...,67.357500,8,0,0,"[0.6975328211998368, 0.011611247046786755, 0.0...","[0.00010825124705981343, 0.0003368525755860405...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[9942036.212268796, 2480196.0696003092, 184539...","[2284746.1115528643, 377360.20508465165, 16474...",0-20
2,0,30,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.111212,0.026408,0.000133,0.137754,0.000793,1.332263,...,66.571358,8,0,0,"[0.6932331149761389, 0.02188893553598149, 0.02...","[4.35309076266402e-05, 5.689478224036808e-05, ...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[116062065.06273228, 19268755.145752113, 78039...","[7451366.027011359, 1235179.7437339195, 554244...",0-30
3,0,40,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.254201,0.025425,0.000133,0.279759,0.002538,0.727116,...,67.223100,8,0,0,"[0.6952150945860903, 0.022138023346156836, 0.0...","[3.790844984280928e-05, 0.00011873273229971483...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[33581900.42937069, 9135692.667925533, 6214825...","[1521753.5429566517, 211334.6357507293, 73895....",0-40
4,0,50,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.513598,0.027100,0.000133,0.540831,0.010062,3.752879,...,65.694834,8,0,0,"[0.6753343408163203, 0.10991481527539418, 0.05...","[0.0014718658263442104, 0.005018048037925155, ...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[240315631.16000515, 38690079.484423034, 15785...","[9216775.957592016, 1477803.267432063, 613338....",0-50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1949,83,19500,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.023502,1.261625,0.280560,1.565687,0.000410,12.430042,...,61.243859,8,68,2,"[0.692921100746221, 0.02529608389693561, 0.017...","[0.0005358212677690198, 0.0017774464360995708,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[249213808.19725654, 129377437.6628547, 794073...","[44387638.56763574, 23377510.277480926, 113380...",83-19500
1950,83,19510,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.031746,1.059493,0.418633,1.509872,0.000447,10.164338,...,60.857965,8,55,1,"[0.6930078452125261, 0.025189933520855814, 0.0...","[0.0004999708375635812, 0.0016692170290463111,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[251543648.34397826, 128170267.73426212, 76487...","[51642854.15924344, 22971675.643509477, 106903...",83-19510
1951,83,19520,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.006026,2.030447,0.087357,2.123830,0.000249,13.582660,...,57.925885,8,76,0,"[0.6916216315011511, 0.02910874889525416, 0.02...","[0.0006670482052557545, 0.0021555228663028567,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[293760342.3192913, 156600578.2904631, 9399497...","[70018460.4124415, 33565732.569601715, 1458086...",83-19520
1952,83,19530,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.020874,2.580934,1.093959,3.695767,0.000520,12.951346,...,60.812323,8,89,8,"[0.6929035677859409, 0.025293613441163654, 0.0...","[0.00042933362534880844, 0.0014389277263746696...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[253674468.7691031, 136021837.73548526, 851748...","[42928437.5251252, 23134389.855043076, 9697096...",83-19530


### Slice best steps

In [33]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',params_lst=[0,1,2],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''

    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1b']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1b']:
            for idx,i in enumerate(params_lst): chi_cols.append(j+'_'+str(idx))
    else: ## Add chi-square for specific label
        assert label in params_lst, "label %s is not in %s"%(label,params_lst)
        label_idx=params_lst.index(label)
        print(label_idx)
        for j in ['chi_1','chi_spec1','chi_1b']: chi_cols.append(j+'_'+str(label_idx))
#     print(chi_cols)
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

# f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=0.65,params_lst=[0.5,0.65,0.8,1.1],use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

In [34]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),
label=ToggleButtons(options=['all']+sigma_lst), params_lst=fixed(sigma_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [35]:
df_sliced=w.result
[int(i.split('-')[1]) for i in df_sliced.label.values]

# df_sliced

[3960, 10280, 11130, 2970, 12410, 3970, 9620, 10210, 6440, 9970]

In [48]:
best_step=[]
df_test=df_merged.copy()
df_test=df_merged[(df_merged.epoch<5000)&(df_merged.epoch>40)]
cut_off=0.9

best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_spec1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_1b',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)

# best_step.append([46669,34281])
best_step=np.unique([i for j in best_step for i in j])
print(best_step)
best_step

[10160 10460 12100 12280 14040 16260 17080 18010 19510]


array([10160, 10460, 12100, 12280, 14040, 16260, 17080, 18010, 19510])

In [49]:
# best_step=[6176]
# best_step= [23985,24570,25155,25740,26325,26910,27495]
# best_step=[int(i.split('-')[1]) for i in df_sliced.label.values]
# best_step=np.arange(40130,40135).astype(int)

In [50]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(9, 61)
[(43, 10160), (44, 10460), (51, 12100), (52, 12280), (59, 14040), (69, 16260), (72, 17080), (76, 18010), (83, 19510)]


In [51]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_1b
1015,43-10160,0.435015,43.132904,0.041426
1045,44-10460,0.336040,29.368707,0.126485
1209,51-12100,0.365233,17.164654,0.032552
1227,52-12280,0.371205,14.493628,0.147877
1403,59-14040,0.367246,21.570874,0.130657
1625,69-16260,2.752037,2.096530,2.482886
1707,72-17080,1.141967,2.127654,0.828925
1800,76-18010,2.160443,1.799358,1.994110
1950,83-19510,2.515030,1.772353,2.004131


### Interactive plot

In [52]:

def f_plot_hist_spec(df,param_labels,sigma_lst,steps_list,bkg_dict,plot_type,img_size):

    assert plot_type in ['hist','spec','grid','spec_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    for par_label in param_labels:
        df=df[df.step.isin(steps_list)]
#         print(df.shape)
        idx=sigma_lst.index(par_label)
        suffix='_%s'%(idx)
        dict_bkg=bkg_dict[str(par_label)]
        
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(par_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))

                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='spec_relative':

                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]
                x2=np.arange(len(y2))

                ### Reference spectrum
                y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
                y=y2/y1
                ## Variance is sum of variance of both variables, since they are uncorrelated

                # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
                yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'+suffix])
                
                plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
                plt.plot(x2, y, marker=marker, linestyle=':',label=label)

            if plot_type=='grid':
                images=np.load(row['fname'+suffix])[:,0,:,:,0]
                print(images.shape)
                f_plot_grid(images[:8],cols=4,fig_size=(8,4))
            
        ### Plot reference data
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
            plt.xlim(0,img_size/2)

        if plot_type=='spec_relative':
            plt.axhline(y=1.0,color='k',linestyle='-.')
            plt.title("Relative spectrum")
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)

    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})


# f_plot_hist_spec(df_merged,[sigma_lst[-1]],sigma_lst,[best_step[0]],bkgnd_dict,'hist')

In [53]:
interact_manual(f_plot_hist_spec,df=fixed(df_best),
                param_labels=SelectMultiple(options=sigma_lst),sigma_lst=fixed(sigma_lst),
                steps_list=SelectMultiple(options=df_best.step.values),
                img_size=fixed(img_size),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='param_labels', options=('0.5', '0.8', '1.1'), value=()), Sel…

<function __main__.f_plot_hist_spec(df, param_labels, sigma_lst, steps_list, bkg_dict, plot_type, img_size)>

In [65]:
best_step

array([30180, 32110, 32530, 32540, 32590, 32600, 32940, 33380])

In [ ]:
sigma=0.5
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_{0}_train_val.npy'.format(sigma)
a1=np.load(fname,mmap_mode='r')[-100:]
a1.shape

images=a1[:,0,:,:,0]
f_plot_grid(images[:8],cols=4,fig_size=(8,4))


### Delete unwanted stored models
(Since deterministic runs aren't working well )

In [ ]:
# # fldr='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210119_134802_cgan_predict_0.65_m2/models'
# fldr=result_dir
# print(fldr)
# flist=glob.glob(fldr+'/models/checkpoint_*.tar')
# len(flist)

In [ ]:
# # Delete unwanted stored images
# for i in flist:
#     try:
#         step=int(i.split('/')[-1].split('_')[-1].split('.')[0])
#         if step not in best_step:
# #             print("Found")
# #             print(step)
#             os.remove(i)
#             pass
#         else: 
#             print(step)
# #             print(i)
#     except Exception as e:
# #         print(e)
# #         print(i)
#         pass

In [ ]:
best_step

In [ ]:
# ! du -hs  /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy

In [8]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/Om0.3_Sg0.5_H70.0.npy'
np.load(fname,mmap_mode='r').shape

(8832, 1, 128, 128, 128)